In [23]:
import imaplib
import email
from email.header import decode_header
import os

# Function to connect to the email and download attachments
def fetch_email_attachment(email_address, password, download_folder):
    # Connect to the Gmail server
    imap = imaplib.IMAP4_SSL("imap.gmail.com")
    imap.login(email_address, password)

    # Select the mailbox you want to check
    imap.select("inbox")

    # Search for all emails
    status, messages = imap.search(None, "ALL")
    messages = messages[0].split()

    # Fetch the latest email
    latest_email = messages[-1]
    res, msg = imap.fetch(latest_email, "(RFC822)")

    for response in msg:
        if isinstance(response, tuple):
            # Parse the raw email
            msg = email.message_from_bytes(response[1])

            # Check if email has attachments
            for part in msg.walk():
                if part.get_content_disposition() == "attachment":
                    filename = part.get_filename()
                    if filename:
                        filename = decode_header(filename)[0][0]
                        if isinstance(filename, bytes):
                            filename = filename.decode()
                        # Save the attachment to the download folder
                        filepath = os.path.join(download_folder, filename)
                        with open(filepath, "wb") as f:
                            f.write(part.get_payload(decode=True))
                        print(f"Downloaded attachment: {filename}")
                        return filepath

    # Close the connection
    imap.logout()
    return None


In [24]:
import import_ipynb
from demo2 import extract_dimensions_with_sizes
from demo2 import process_dimensions
from demo2 import save_to_mongodb
from demo2 import save_to_excel_formatted
from demo2 import extract_spec_sheet_data
from demo2 import save_spec_sheet_to_mongo
from demo2 import save_to_excel
import pandas as pd
from demo2 import extract_image_from_pdf
from demo2 import generate_pdf_from_excel_and_properties
# Main Function
def main():
    # Email credentials
    email_address = "abisheak9363@gmail.com"
    password = "hvhw yxee inpo qwdx"  # Replace with your app password (not raw password)

    # Folder to save the downloaded PDF
    download_folder = "./downloads"
    os.makedirs(download_folder, exist_ok=True)

    # Fetch the attachment
    pdf_path = fetch_email_attachment(email_address, password, download_folder)

    if pdf_path:
        print(f"PDF file ready for processing: {pdf_path}")
        
        # Pass the PDF to the existing code's main function
        output_excel_path = "output_formatted.xlsx"
        db_name = "PDFData_sample"
        collection_name = "DynamicDimensionsNew"

        # Extract dimensions dynamically
        dimensions, headers = extract_dimensions_with_sizes(pdf_path)

        # Process and structure dimensions
        dataframe = process_dimensions(dimensions, headers)

        # Save to MongoDB
        save_to_mongodb(dataframe, db_name, collection_name)

        # Save to Excel with formatting
        save_to_excel_formatted(dataframe, output_excel_path)
        
        #####SPEC SHEET  ########################
        
        
        spec_sheet_data = extract_spec_sheet_data(pdf_path)

        # Save to MongoDB
        save_spec_sheet_to_mongo(spec_sheet_data)

        # Save to Excel
        if spec_sheet_data:
            df = pd.DataFrame(spec_sheet_data)
            output_file = "spec_sheet_output.xlsx"
            save_to_excel(df, output_file)
            
        image_path = extract_image_from_pdf(pdf_path)
        generate_pdf_from_excel_and_properties(output_file,"spec_final.pdf",pdf_path)
        
        print("PDF processing completed!")
    else:
        print("No PDF attachment found.")
        
    
    

if __name__ == "__main__":
    main()


Downloaded attachment: input_doc.pdf
PDF file ready for processing: ./downloads\input_doc.pdf
Data saved to MongoDB database 'PDFData_sample', collection 'DynamicDimensionsNew'
Formatted data successfully saved to output_formatted.xlsx
8 records saved to MongoDB collection 'spec_sheet'.
Spec Sheet data successfully saved to spec_sheet_output.xlsx


<string>:61: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.


PDF generated successfully: spec_final.pdf
PDF processing completed!
